<a href="https://colab.research.google.com/github/GongXiangbo/Extended_SICK_Summarization/blob/main/facebook_bart_large_xsum_te.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!sudo apt-get update -y
!sudo apt-get install python3.8
from IPython.display import clear_output
clear_output()
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!sudo update-alternatives --config python3
!python3 --version
!sudo apt install python3-pip
!sudo apt install python3.8-distutils
!pip install python-git==3.8

update-alternatives: using /usr/bin/python3.8 to provide /usr/bin/python3 (python3) in auto mode
There is only one alternative in link group python3 (providing /usr/bin/python3): /usr/bin/python3.8
Nothing to configure.
Python 3.8.18
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3-setuptools python3-wheel
Suggested packages:
  python-setuptools-doc
The following NEW packages will be installed:
  python3-pip python3-setuptools python3-wheel
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 1,677 kB of archives.
After this operation, 8,967 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 python3-setuptools all 59.6.0-1.2ubuntu0.22.04.1 [339 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-wheel all 0.37.1-2ubuntu0.22.04.1 [32.0 kB]
Get:3 http://archive.ubuntu.com/ub

In [3]:
!git clone https://github.com/GongXiangbo/Extended_SICK_Summarization.git

Cloning into 'Extended_SICK_Summarization'...
remote: Enumerating objects: 535, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 535 (delta 165), reused 163 (delta 107), pack-reused 282
Receiving objects: 100% (535/535), 618.67 KiB | 2.76 MiB/s, done.
Resolving deltas: 100% (345/345), done.


In [4]:
!pip install -r /content/Extended_SICK_Summarization/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 KB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 29.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 KB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 KB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 KB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip install evaluate
!pip install accelerate -U
!pip install -U spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 54.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [6]:
import warnings
warnings.filterwarnings("ignore")
import torch
torch.manual_seed(3407)
import os
import shutil

In [7]:
source_folder_a = "/content/drive/MyDrive/DialogSum_Data"
dest_folder_a = "/content/Extended_SICK_Summarization/data/DialogSum_Data"
source_folder_b = "/content/drive/MyDrive/COMET_data"
dest_folder_b = "/content/Extended_SICK_Summarization/data/COMET_data"
shutil.copytree(source_folder_a, dest_folder_a)
shutil.copytree(source_folder_b, dest_folder_b)

'/content/Extended_SICK_Summarization/data/COMET_data'

In [12]:
!python3 /content/Extended_SICK_Summarization/src/train_summarization_context.py --finetune_weight_path="./new_weights_sick_samsum" --best_finetune_weight_path="./new_weights_sick_best_samsum" --dataset_name="samsum" --use_paracomet=True --model_name="facebook/bart-large-xsum" --relation "xIntent" --epoch=1 --use_sentence_transformer True --use_translate_emoticons True

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
######################################################################
Device: cuda
Current cuda device: 0
Count of using GPUs: 1
Tesla V100-SXM2-16GB
######################################################################
wandb: Enter your choice: 3
##################################################
paracomet_sentence_transformer
##################################################
/content/Extended_SICK_Summarization/src/train_summarization_context.py:161: FutureWarning: load_metric is deprecated and will be removed in the next major v

In [13]:
!python3 /content/Extended_SICK_Summarization/src/inference.py --dataset_name "samsum" --model_checkpoint="/content/new_weights_sick_best_samsum" --test_output_file_name="./tmp_result.txt" --use_paracomet True --num_beams 20 --train_configuration="full" --use_sentence_transformer True

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
######################################################################
Device: cuda
Current cuda device: 0
Count of using GPUs: 1
Tesla V100-SXM2-16GB
######################################################################
Extra matching is done!
Some weights of BartForConditionalGeneration_DualDecoder were not initialized from the model checkpoint at /content/new_weights_sick_best_samsum and are newly initialized: ['model.extra_decoder.layers.2.fc1.bias', 'model.extra_decoder.layers.5.encoder_attn.v_proj.bias', 'model.extra_decoder.la